![Two data scientists working on a dashboard.](hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `student_id`   | A unique ID for each student.                 |
| `city`  | A code for the city the student lives in.  |
| `city_development_index` | A scaled development index for the city.       |
| `gender` | The student's gender.       |
| `relevant_experience` | An indicator of the student's work relevant experience.       |
| `enrolled_university` | The type of university course enrolled in (if any).       |
| `education_level` | The student's education level.       |
| `major_discipline` | The educational discipline of the student.       |
| `experience` | The student's total work experience (in years).       |
| `company_size` | The number of employees at the student's current employer.       |
| `last_new_job` | The number of years between the student's current and previous jobs.       |
| `training_hours` | The number of hours of training completed.       |
| `job_change` | An indicator of whether the student is looking for a new job (`1`) or not (`0`).       |

In [79]:
# Start your code here!
import pandas as pd

# Read file 
ds_jobs = pd.read_csv('customer_train.csv')
display(ds_jobs.head(), ds_jobs.shape, ds_jobs.info(), ds_jobs.memory_usage())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  int64  
dtypes: float64(1), int64(3), object(10)
me

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0
3,33241,city_115,0.789,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1
4,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0


(19158, 14)

None

Index                        128
student_id                153264
city                      153264
city_development_index    153264
gender                    153264
relevant_experience       153264
enrolled_university       153264
education_level           153264
major_discipline          153264
experience                153264
company_size              153264
company_type              153264
last_new_job              153264
training_hours            153264
job_change                153264
dtype: int64

In [80]:
# Create ds_jobs_clean to compare after
ds_jobs_clean = ds_jobs.copy()

In [81]:
# Inspect obj columns to check if it is ordered or unordered categorical data
obj_cols = ds_jobs_clean.select_dtypes(include=['object'])

# Check value counts for each column in a easy to see way
for col in obj_cols.columns:
    print(f'\ncolumn: {col}')
    print(obj_cols[col].value_counts(dropna=False))


column: city
city_103    4355
city_21     2702
city_16     1533
city_114    1336
city_160     845
            ... 
city_129       3
city_111       3
city_121       3
city_140       1
city_171       1
Name: city, Length: 123, dtype: int64

column: gender
Male      13221
NaN        4508
Female     1238
Other       191
Name: gender, dtype: int64

column: relevant_experience
Has relevant experience    13792
No relevant experience      5366
Name: relevant_experience, dtype: int64

column: enrolled_university
no_enrollment       13817
Full time course     3757
Part time course     1198
NaN                   386
Name: enrolled_university, dtype: int64

column: education_level
Graduate          11598
Masters            4361
High School        2017
NaN                 460
Phd                 414
Primary School      308
Name: education_level, dtype: int64

column: major_discipline
STEM               14492
NaN                 2813
Humanities           669
Other                381
Business Degree

In [82]:
# Convert int64 to int32
int_cols = ['student_id', 'training_hours', 'job_change']
ds_jobs_clean[int_cols] = ds_jobs_clean[int_cols].astype('int32')

# Convert float64 to float16
float_cols = ['city_development_index']
ds_jobs_clean[float_cols] = ds_jobs_clean[float_cols].astype('float16')

# Convert unordered categorical to category dtype
cat_cols = ['city', 'gender', 'major_discipline', 'company_type']
ds_jobs_clean[cat_cols] = ds_jobs_clean[cat_cols].astype('category')

In [83]:
# Create ordered categorical dtypes

# relevant_experience
rel_order = ['No relevant experience', 'Has relevant experience']
rel_dtype = pd.CategoricalDtype(categories=rel_order, ordered=True)

# enrolled_university
enr_order = ['no_enrollment', 'Part time course', 'Full time course']
enr_dtype = pd.CategoricalDtype(categories=enr_order, ordered=True)

# education_level
education_order = ['Primary School', 'High School', 'Masters', 'Phd', 'Graduate']
ed_dtype = pd.CategoricalDtype(categories=education_order, ordered=True)

# experience
exp_order = ['<1', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '>20']
exp_dtype = pd.CategoricalDtype(categories=exp_order, ordered=True)

# company_size
comp_order = ['<10', '10-49', '50-99', '100-499', '500-999', '1000-4999', '5000-9999', '10000+']
comp_dtype = pd.CategoricalDtype(categories=comp_order, ordered=True)

# last_new_job
last_order = ['never', '<1', '1', '2', '3', '4', '>4']
last_dtype = pd.CategoricalDtype(categories=last_order, ordered=True)


In [84]:
# Convert columns to ordered categorical dtype

ds_jobs_clean['relevant_experience'] = ds_jobs_clean['relevant_experience'].astype(rel_dtype)

ds_jobs_clean['enrolled_university'] = ds_jobs_clean['enrolled_university'].astype(enr_dtype)

ds_jobs_clean['education_level'] = ds_jobs_clean['education_level'].astype(ed_dtype)

ds_jobs_clean['experience'] = ds_jobs_clean['experience'].astype(exp_dtype)

ds_jobs_clean['company_size'] = ds_jobs_clean['company_size'].astype(comp_dtype)

ds_jobs_clean['last_new_job'] = ds_jobs_clean['last_new_job'].astype(last_dtype)

In [85]:
# Check memory usage difference
print('Memory usage in bytes')
display(ds_jobs.memory_usage().sum(),ds_jobs_clean.memory_usage().sum())

# Check ds_jobs_clean info
display(ds_jobs_clean.info())

Memory usage in bytes


2145824

467564

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   student_id              19158 non-null  int32   
 1   city                    19158 non-null  category
 2   city_development_index  19158 non-null  float16 
 3   gender                  14650 non-null  category
 4   relevant_experience     19158 non-null  category
 5   enrolled_university     18772 non-null  category
 6   education_level         18698 non-null  category
 7   major_discipline        16345 non-null  category
 8   experience              19093 non-null  category
 9   company_size            13220 non-null  category
 10  company_type            13018 non-null  category
 11  last_new_job            18735 non-null  category
 12  training_hours          19158 non-null  int32   
 13  job_change              19158 non-null  int32   
dtypes: category(10), float

None

In [86]:
# Filter the ds_jobs_clean dataframe to requested parameters

ds_jobs_clean = ds_jobs_clean[(ds_jobs_clean['experience'] >= '10') & (ds_jobs_clean['company_size'] >= '1000-4999')]